In [66]:
# Импортируем библиотеки 
import os
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [67]:
# Отметим необходимые пути
task_path = 'tasks/Merch_CB_hack.csv'
test_path = 'tasks/merch_cb_hack_8_9.csv'

In [68]:
# Пофиксим файл с данными
with open(task_path, 'r', encoding='utf-8') as file:
    file_content = file.read()
if ', интернет' in file_content:
    file_content = file_content.replace(', интернет', ' интернет')
    with open(task_path, 'w', encoding='utf-8') as file:
        file.write(file_content)
    

In [69]:
# Прочитаем данные
df = pd.read_csv(task_path, sep=',', encoding='utf-8')

In [70]:
# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [71]:
# Удаляем месяц, так как он не несет никакой информации
df = df.drop('month', axis=1)

In [72]:
# Удалим строки с merchant_name, которое встречается меньше 7 раз в df
df = df.groupby('merchant_name').filter(lambda x : len(x) > 6)

In [73]:
# Построим функцию z-нормализации
def z_normalization(x, mean, std):
    return (x - mean) / std

# И обратную функцию
def z_reverse(x, mean, std):
    return x * std + mean

In [74]:
# Для каждого merchant_name найдем среднее и стандартное отклонение cashback 
mean_cashback = df.groupby('merchant_name')['cashback'].mean()
std_cashback = df.groupby('merchant_name')['cashback'].std()

# И запишем в виде словаря
cashback_func = {}
for merchant_name in df['merchant_name'].unique():
    cashback_func[merchant_name] = (mean_cashback[merchant_name], std_cashback[merchant_name])

In [75]:
# Добавим в датасет нормализованное значение cashback
df['normalized_cashback'] = df.apply(lambda row: z_normalization(row['cashback'], cashback_func[row['merchant_name']][0], cashback_func[row['merchant_name']][1]), axis=1)

In [76]:
# Построим функцию для поиска MCC кода
def parse_mcc(organisation):
    request = requests.get(f'https://mcc-codes.ru/search/?q={'+'.join(organisation.split())}')

In [77]:
# Сохраним датасет
if os.path.exists('data.csv'):
    os.remove('data.csv')
df.to_csv('data.csv', index=False, encoding='utf-8')